<div class='alert alert-warning'>

NumPy's interactive examples are experimental and may not always work as expected, with high load times especially on low-resource platforms, and the version of NumPy might not be in sync with the one you are browsing the documentation for. If you encounter any issues, please report them on the [NumPy issue tracker](https://github.com/numpy/numpy/issues).

</div>

Here is how we might write an ``iter_add`` function, using the
Python iterator protocol:


In [ ]:
import numpy as np

In [ ]:
def iter_add_py(x, y, out=None):
    addop = np.add
    it = np.nditer([x, y, out], [],
                [['readonly'], ['readonly'], ['writeonly','allocate']])
    with it:
        for (a, b, c) in it:
            addop(a, b, out=c)
        return it.operands[2]

Here is the same function, but following the C-style pattern:


In [ ]:
def iter_add(x, y, out=None):
   addop = np.add
   it = np.nditer([x, y, out], [],
               [['readonly'], ['readonly'], ['writeonly','allocate']])
   with it:
       while not it.finished:
           addop(it[0], it[1], out=it[2])
           it.iternext()
       return it.operands[2]

Here is an example outer product function:


In [ ]:
def outer_it(x, y, out=None):
    mulop = np.multiply
    it = np.nditer([x, y, out], ['external_loop'],
            [['readonly'], ['readonly'], ['writeonly', 'allocate']],
            op_axes=[list(range(x.ndim)) + [-1] * y.ndim,
                     [-1] * x.ndim + list(range(y.ndim)),
                     None])
    with it:
        for (a, b, c) in it:
            mulop(a, b, out=c)
        return it.operands[2]

In [ ]:
a = np.arange(2)+1
b = np.arange(3)+1
outer_it(a,b)

array([[1, 2, 3],
       [2, 4, 6]])

Here is an example function which operates like a "lambda" ufunc:


In [ ]:
def luf(lamdaexpr, *args, **kwargs):
   '''luf(lambdaexpr, op1, ..., opn, out=None, order='K', casting='safe', buffersize=0)'''
   nargs = len(args)
   op = (kwargs.get('out',None),) + args
   it = np.nditer(op, ['buffered','external_loop'],
           [['writeonly','allocate','no_broadcast']] +
                           [['readonly','nbo','aligned']]*nargs,
           order=kwargs.get('order','K'),
           casting=kwargs.get('casting','safe'),
           buffersize=kwargs.get('buffersize',0))
   while not it.finished:
       it[0] = lamdaexpr(*it[1:])
       it.iternext()
   return it.operands[0]

In [ ]:
a = np.arange(5)
b = np.ones(5)
luf(lambda i,j:i*i + j/2, a, b)

array([  0.5,   1.5,   4.5,   9.5,  16.5])

If operand flags ``"writeonly"`` or ``"readwrite"`` are used the
operands may be views into the original data with the
`WRITEBACKIFCOPY` flag. In this case `nditer` must be used as a
context manager or the `nditer.close` method must be called before
using the result. The temporary data will be written back to the
original data when the :meth:`~object.__exit__` function is called
but not before:


In [ ]:
a = np.arange(6, dtype=np.int32)[::-2]
with np.nditer(a, [],
       [['writeonly', 'updateifcopy']],
       casting='unsafe',
       op_dtypes=[np.dtype(np.float32)]) as i:
   x = i.operands[0]
   x[:] = [-1, -2, -3]
   # a still unchanged here
a, x

(array([-1, -2, -3], dtype=int32), array([-1., -2., -3.], dtype=float32))

It is important to note that once the iterator is exited, dangling
references (like `x` in the example) may or may not share data with
the original data `a`. If writeback semantics were active, i.e. if
`x.base.flags.writebackifcopy` is `True`, then exiting the iterator
will sever the connection between `x` and `a`, writing to `x` will
no longer write to `a`. If writeback semantics are not active, then
`x.data` will still point at some part of `a.data`, and writing to
one will affect the other.

Context management and the `close` method appeared in version 1.15.0.